In [2]:
import pandas as pd
import json
import numpy as np

In [3]:
with open("trial_task.json", "r") as f:
    data = f.read()
    json_data = json.loads(data)

In [4]:
orders_data = pd.read_json("trial_task.json") 
orders_data.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [5]:
d = {'product': [], 'price': [], 'quantity': [], 'order_id': []}
result = pd.DataFrame(data=d)
for i in json_data:
    example = i["products"]
    r = pd.DataFrame(data=example)
    r['order_id'] = [i["order_id"] for _ in range(np.shape(r)[0])]
    result = pd.concat([result, r], ignore_index=True)
result['order_id'] = result['order_id'].astype(int)
result['quantity'] = result['quantity'].astype(int)
result['price'] = result['price'].astype(int)
result.head()

,product,price,quantity,order_id
0,ломтик июльского неба,450,1,11973
1,билет в Израиль,1000,3,11973
2,статуэтка Ленина,200,3,11973
3,билет в Израиль,1000,1,62239
4,зеленая пластинка,10,2,85794


In [6]:
orders_data = orders_data.set_index('order_id').join(result.set_index('order_id'), on='order_id')\
                            .drop(['products'], axis=1)
orders_data['total_cost_for_each_product'] = orders_data['price'] * orders_data['quantity']
orders_data = orders_data.reset_index(inplace=False)
orders_data.head()

,order_id,warehouse_name,highway_cost,product,price,quantity,total_cost_for_each_product
0,11973,Мордор,-70,ломтик июльского неба,450,1,450
1,11973,Мордор,-70,билет в Израиль,1000,3,3000
2,11973,Мордор,-70,статуэтка Ленина,200,3,600
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1,1000
4,85794,отель Лето,-50,зеленая пластинка,10,2,20


### 1. Найти тариф стоимости доставки для каждого склада.

In [7]:
warehouses = orders_data['warehouse_name'].unique()
print(warehouses)

['Мордор' 'хутор близ Диканьки' 'отель Лето' 'остров невезения'
 'гиперборея']


In [8]:
d1 = {'warehouse_name' : warehouses, 'shipping_cost' : np.zeros(len(warehouses))}
shipping_cost_rate = pd.DataFrame(data=d1)
shipping_cost_rate

,warehouse_name,shipping_cost
0,Мордор,0.0
1,хутор близ Диканьки,0.0
2,отель Лето,0.0
3,остров невезения,0.0
4,гиперборея,0.0


In [9]:
for i in range(len(warehouses)):
    sum_cost = orders_data[orders_data['warehouse_name'] == warehouses[i]].groupby('order_id', as_index=False)\
                    .aggregate({'highway_cost' : 'mean', 'quantity' : 'sum'})\
                        .sum().drop(['order_id'])
    shipping_cost_rate.at[i, 'shipping_cost'] = sum_cost['highway_cost'] / sum_cost['quantity']

In [10]:
shipping_cost_rate  #тариф стоимости доставки для каждого склада

,warehouse_name,shipping_cost
0,Мордор,-10.0
1,хутор близ Диканьки,-15.0
2,отель Лето,-25.0
3,остров невезения,-5.0
4,гиперборея,-20.0


### 2. Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit').

In [40]:
orders_data1 = orders_data.join(shipping_cost_rate.set_index('warehouse_name'), on='warehouse_name')
orders_data1['shipping_cost_for_each_product'] = orders_data1['shipping_cost'] * orders_data1['quantity']
orders_data1 = orders_data1.drop(['highway_cost', 'shipping_cost'], axis=1)
orders_data1.head()

,order_id,warehouse_name,product,price,quantity,total_cost_for_each_product,shipping_cost_for_each_product
0,11973,Мордор,ломтик июльского неба,450,1,450,-10.0
1,11973,Мордор,билет в Израиль,1000,3,3000,-30.0
2,11973,Мордор,статуэтка Ленина,200,3,600,-30.0
3,62239,хутор близ Диканьки,билет в Израиль,1000,1,1000,-15.0
4,85794,отель Лето,зеленая пластинка,10,2,20,-50.0


In [42]:
products_information = orders_data1.groupby('product', as_index=False)\
            .aggregate({'quantity' : 'sum', 'total_cost_for_each_product' : 'sum'\
                        , 'shipping_cost_for_each_product' : 'sum'})
products_information = products_information.rename({'total_cost_for_each_product' : 'income'\
                            , 'shipping_cost_for_each_product' : 'expenses'}, axis=1)
products_information['profit'] = (products_information['income'] + products_information['expenses']).astype(int)
products_information

,product,quantity,income,expenses,profit
0,автограф Стаса Барецкого,48,28800,-820.0,27980
1,билет в Израиль,58,58000,-1050.0,56950
2,зеленая пластинка,61,610,-920.0,-310
3,ломтик июльского неба,42,18900,-670.0,18230
4,плюмбус,65,16250,-940.0,15310
5,подписка на suppi-блог,33,4950,-545.0,4405
6,статуэтка Ленина,68,13600,-935.0,12665


### 3. Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов.

In [61]:
order_id_profit = orders_data.groupby('order_id', as_index=False)\
            .aggregate({'highway_cost' : 'mean','total_cost_for_each_product' : 'sum'})
order_id_profit['order_profit'] = (order_id_profit['total_cost_for_each_product']\
                                             + order_id_profit['highway_cost']).astype(int)
order_id_profit = order_id_profit.drop(['highway_cost', 'total_cost_for_each_product'], axis=1)
order_id_profit.head(10)

,order_id,order_profit
0,124,705
1,1391,490
2,2091,1300
3,2108,200
4,2558,355
5,2751,-40
6,2930,720
7,4035,4125
8,5430,1120
9,6535,2325


### 4. Составить табличку типа 'warehouse_name' , 'product', 'quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада).

In [72]:
percent_profit_for_product_of_warehouse = orders_data1.groupby(['warehouse_name', 'product'], as_index=False)\
            .aggregate({'shipping_cost_for_each_product' : 'sum', 'total_cost_for_each_product' : 'sum'})
percent_profit_for_product_of_warehouse.head()

,warehouse_name,product,shipping_cost_for_each_product,total_cost_for_each_product
0,Мордор,автограф Стаса Барецкого,-40.0,2400
1,Мордор,билет в Израиль,-90.0,9000
2,Мордор,зеленая пластинка,-110.0,110
3,Мордор,ломтик июльского неба,-30.0,1350
4,Мордор,плюмбус,-60.0,1500
